# Autogluon을 활용한 앙상블 모델링 (California Housing)

이 노트북은 `Plus_7_ensemble.ipynb`를 기반으로 하며, **Autogluon** 라이브러리를 사용하여 다음 모델들에 대한 학습, 튜닝 및 앙상블을 수행합니다.

- **단일 모델**: GBM (LightGBM), XGBoost, CatBoost, Random Forest
- **앙상블**: Voting (Weighted Ensemble), Stacking

## 1. 환경 설정 및 데이터 로드

In [ ]:
# AutoGluon 설치
# !pip install autogluon

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

np.set_printoptions(precision=4)

### 데이터셋 가져오기

In [ ]:
data = fetch_california_housing()
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['MedHouseVal'] = data['target']

df.head()

### 학습/테스트 데이터 분리
Autogluon의 성능 평가를 위해 데이터를 학습용(Training)과 테스트용(Test)으로 분리합니다.

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training Data Size: {train_data.shape}")
print(f"Test Data Size: {test_data.shape}")

## 2. Autogluon 모델 학습

요청하신 다음 모델들을 포함하여 학습을 진행합니다:
- **GBM** (Gradient Boosting Machine, 주로 LightGBM)
- **XGB** (XGBoost)
- **CAT** (CatBoost)
- **RF** (Random Forest)

또한 `presets='best_quality'` 설정을 사용하여 **Stacking**과 **Weighted Ensemble(Voting)**을 자동으로 수행하도록 합니다.

In [ ]:
# 학습할 모델의 하이퍼파라미터 지정
# Autogluon에서는 'GBM'이 LightGBM 역할을 합니다.
hyperparameters = {
    'GBM': {}, 
    'XGB': {}, 
    'CAT': {}, 
    'RF': {}
}

# TabularPredictor 설정
predictor = TabularPredictor(
    label='MedHouseVal', 
    eval_metric='rmse',  # 회귀 문제이므로 RMSE 사용
    problem_type='regression'
).fit(
    train_data,
    hyperparameters=hyperparameters,
    presets='best_quality',  # Stacking 및 고성능 앙상블 활성화
    time_limit=300  # 시간 제한 설정 (초 단위, 필요에 따라 조절 가능 - 예: 3600)
)

## 3. 결과 확인 (Leaderboard)

학습된 개별 모델 및 앙상블 모델(WeightedEnsemble, Stacking)의 성능을 확인합니다.

In [ ]:
predictor.leaderboard(test_data, silent=False)

## 4. 테스트 데이터 예측 및 평가

In [ ]:
# 테스트 데이터에 대한 성능 평가
perf = predictor.evaluate(test_data)

# 예측 값 생성
y_pred = predictor.predict(test_data.drop(columns=['MedHouseVal']))

print("Performance on Test Data:")
print(perf)
print("\nFirst 5 predictions:")
print(y_pred.head())

## 5. 모델 세부 정보
최적의 모델(보통 WeightedEnsemble)에 대한 정보를 확인합니다.

In [ ]:
# 최적 모델 확인
print(predictor.get_model_best())

In [ ]:
# 각 모델의 특성 중요도 (Feature Importance) - Optional
# 시간이 오래 걸릴 수 있으므로 필요 시 주석 해제하여 실행
# feature_importance = predictor.feature_importance(test_data)
# print(feature_importance)